In [1]:
import vgg16
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from keras.layers import TimeDistributed, Activation
from numpy.random import choice

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()#.lower()

In [3]:
print(len(text))
print(text[:100])

600901
PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all ph


In [4]:
!tail -n25 {path}

are thinkers who believe in the saints.


144

It stands to reason that this sketch of the saint, made upon the model
of the whole species, can be confronted with many opposing sketches that
would create a more agreeable impression. There are certain exceptions
among the species who distinguish themselves either by especial
gentleness or especial humanity, and perhaps by the strength of their
own personality. Others are in the highest degree fascinating because
certain of their delusions shed a particular glow over their whole
being, as is the case with the founder of christianity who took himself
for the only begotten son of God and hence felt himself sinless; so that
through his imagination--that should not be too harshly judged since the
whole of antiquity swarmed with sons of god--he attained the same goal,
the sense of complete sinlessness, complete irresponsibility, that can
now be attained by every individual through science.--In the same manner
I have viewed the saints of India

In [5]:
chars = sorted(list(set(text)))
chars.insert(0, '\0') # Make the index starting from 1
print(chars)
print(len(chars))
''.join(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x86', '\xa4', '\xa6', '\xa9', '\xab', '\xc3']
86


'\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz\x86\xa4\xa6\xa9\xab\xc3'

In [6]:
char_to_index = {c:i for i, c in enumerate(chars)}
index_to_char = {i:c for i, c in enumerate(chars)}

In [7]:
indexes = [char_to_index[c] for c in text]

In [8]:
''.join([index_to_char[i] for i in indexes[:50]])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what th'

In [9]:
max_len = 40
sentences = []
next_chars = []
for i in range(len(indexes)- max_len):
    sentences.append(indexes[i:i+max_len])
    next_chars.append(indexes[i+1:i+max_len+1])

In [10]:
print(len(sentences))
print(len(next_chars))
#print(text[:50])
print("%s\n%s"%(sentences[-1], next_chars[-1]))

600861
600861
[41, 2, 50, 42, 45, 39, 31, 2, 28, 46, 2, 47, 35, 32, 2, 36, 41, 31, 36, 30, 28, 47, 36, 42, 41, 46, 2, 42, 33, 2, 46, 36, 41, 33, 48, 39, 41, 32, 46, 46]
[2, 50, 42, 45, 39, 31, 2, 28, 46, 2, 47, 35, 32, 2, 36, 41, 31, 36, 30, 28, 47, 36, 42, 41, 46, 2, 42, 33, 2, 46, 36, 41, 33, 48, 39, 41, 32, 46, 46, 10]


In [11]:
#[np.array(s) for s in sentences]
#m = np.concatenate([np.array([s for s in ss]) for ss in sentences])
sentences1 = np.row_stack([[s for s in ss] for ss in sentences])
next_chars1 = np.row_stack([[s for s in ss] for ss in next_chars])
print(sentences1.shape)
print(next_chars1.shape)

(600861, 40)
(600861, 40)


In [12]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]])
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [13]:
print(sentences.shape)
print(next_chars.shape)

(600859, 40)
(600859, 40)


In [14]:
np.array(sentences[0])

array([43, 45, 32, 33, 28, 30, 32,  1,  1,  1, 46, 48, 43, 43, 42, 46, 36, 41, 34,  2, 47, 35, 28,
       47,  2, 47, 45, 48, 47, 35,  2, 36, 46,  2, 28,  2, 50, 42, 40, 28])

In [16]:
print(next_chars.shape)

(600859, 40)


In [195]:
sentences1[0]

array(['p', 'r', 'e', 'f', 'a', 'c', 'e', '\n', '\n', '\n', 's', 'u', 'p', 'p', 'o', 's', 'i', 'n',
       'g', ' ', 't', 'h', 'a', 't', ' ', 't', 'r', 'u', 't', 'h', ' ', 'i', 's', ' ', 'a', ' ',
       'w', 'o', 'm', 'a'],
      dtype='|S1')

In [52]:
n_fac = 24
vocab_size = len(chars)
print(vocab_size)

86


In [23]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_to_index[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [19]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=max_len),
        LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size)),
        Activation('softmax')
    ])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, recurrent_dropout=0.2, implementation=2, dropout=0.2, input_shape=(None, 24), return_sequences=True)`
/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, recurrent_dropout=0.2, implementation=2, dropout=0.2, return_sequences=True)`


In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [21]:
model.fit(sentences1, np.expand_dims(next_chars1,-1), batch_size=64, nb_epoch=1)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
600861/600861 [==============================] - 2618s 4ms/step - loss: 1.4950


In [24]:
print_example()

ethics is a basic foundation of all that is seldom fundamental tendency than complex to the free soul, in the metaphysicians, and
here and should aver to exist and as. finally, granting to
which i had not shall be the most delicate and
melanchology of will.--"inspires his "one widen" feels him (are something highly
believed in everything problem of man. a
wi


# 3-Char Model

In [10]:
cs = 3
c1_dat = [indexes[i] for i in range(0, len(indexes)-cs-1, cs)]
c2_dat = [indexes[i+1] for i in range(0, len(indexes)-cs-1, cs)]
c3_dat = [indexes[i+2] for i in range(0, len(indexes)-cs-1, cs)]
c4_dat = [indexes[i+3] for i in range(0, len(indexes)-cs-1, cs)]

In [11]:
#c1_dat = [indexes[i] for i in xrange(0, len(indexes)-1-cs, cs)]
print(len(c1_dat), len(c2_dat), len(c3_dat), len(c4_dat))
print(indexes[:10])
print(c1_dat[0:10])
print(c2_dat[0:10])
print(c3_dat[0:10])
print(c4_dat[0:10])


200299 200299 200299 200299
[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]
[40, 30, 29, 1, 40, 43, 31, 61, 2, 74]
[42, 25, 1, 43, 40, 33, 2, 54, 44, 73]
[29, 27, 1, 45, 39, 38, 73, 73, 71, 61]
[30, 29, 1, 40, 43, 31, 61, 2, 74, 2]


In [12]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)
y = np.stack(c4_dat)

In [13]:
print(x1.shape)
print(x2.shape)
print(x3.shape)
print(y.shape)
print(x1[0:5])

(200299,)
(200299,)
(200299,)
(200299,)
[40 30 29  1 40]


In [14]:
x1[:4], x2[:4], x3[:4], y[:4]

(array([40, 30, 29,  1]),
 array([42, 25,  1, 43]),
 array([29, 27,  1, 45]),
 array([30, 29,  1, 40]))

In [20]:
n_fac = 42
vocab_size = len(chars)

In [21]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name = name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [22]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

In [23]:
n_hidden = 256
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='tanh')

c1_hidden = dense_in(c1)
hidden_2 = dense_hidden(c1_hidden)
c2_dense = dense_in(c2)
c2_hidden = merge([c2_dense, hidden_2])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [24]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden=merge([c3_dense, hidden_3])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()


In [25]:
dense_out = Dense(vocab_size, activation='softmax')
c4_out = dense_out(c3_hidden)


In [26]:
model = Model([c1_in, c2_in, c3_in], c4_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


In [27]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/4
200299/200299 [==============================] - 12s 60us/step - loss: 2.4000
Epoch 2/4
200299/200299 [==============================] - 12s 60us/step - loss: 2.2638
Epoch 3/4
200299/200299 [==============================] - 12s 61us/step - loss: 2.2145
Epoch 4/4
200299/200299 [==============================] - 12s 60us/step - loss: 2.1711


In [88]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c3 (InputLayer)                 (None, 1)            0                                            
__________________________________________________________________________________________________
c2 (InputLayer)                 (None, 1)            0                                            
__________________________________________________________________________________________________
c1 (InputLayer)                 (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 24)        2064        c3[0][0]                         
__________________________________________________________________________________________________
embedding_

In [94]:
def get_next(inp):
    idxs = [char_to_index[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [81]:
get_next('phi')

'l'

In [82]:
get_next(' th')

'e'

In [87]:
get_next('cha')

'l'

# 8-Char Model

In [28]:
cs = 8
c_in_dat = [[indexes[i+n] for i in range(0, len(indexes)-cs-1, cs)] for n in range(cs)]

In [34]:
[len(c_in_dat[n]) for n in range(cs)]
[c_in_dat[n][:10] for n in range(cs)]

[[40, 1, 33, 2, 72, 67, 73, 2, 68, 57],
 [42, 1, 38, 44, 2, 9, 61, 73, 73, 1],
 [29, 43, 31, 71, 54, 9, 58, 61, 2, 59],
 [30, 45, 2, 74, 2, 76, 67, 58, 60, 68],
 [25, 40, 73, 73, 76, 61, 24, 71, 71, 71],
 [27, 40, 61, 61, 68, 54, 2, 58, 68, 2],
 [29, 39, 54, 2, 66, 73, 33, 2, 74, 72],
 [1, 43, 73, 62, 54, 2, 72, 67, 67, 74]]

In [35]:
c_out_dat = [indexes[i+cs] for i in range(0, len(indexes)-cs-1, cs)]
print(c_out_dat[:10])

[1, 33, 2, 72, 67, 73, 2, 68, 57, 72]


In [70]:
xs = [np.stack(c[:-2]) for c in c_in_dat]
y = np.stack(c_out_dat[:-2])

In [71]:
print(len(xs))
print([xs[n].shape for n in range(cs)])
print(y.shape)

8
[(75110,), (75110,), (75110,), (75110,), (75110,), (75110,), (75110,), (75110,)]
(75110,)


In [84]:
n_fac = 42
print(vocab_size)

86


In [73]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name = name + '_in')
    emb = Embedding(n_in, n_out, input_length=1, name = name + '_emb')(inp)
    return inp, Flatten()(emb)

In [74]:
c_ins = [embedding_input('c' + str(n), vocab_size, n_fac) for n in range(cs)]

In [75]:
n_hidden = 256

In [85]:
dense_in = Dense(n_hidden, activation = "relu")
dense_hidden = Dense(n_hidden, activation = 'relu', init = "identity")
dense_out = Dense(vocab_size, activation = "softmax") #Why do we need softmax here? 

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="identity", activation="relu")`
  from ipykernel import kernelapp as app


In [86]:
print(type(c_ins[0][0]), type(c_ins[0][1]))

<class 'theano.tensor.var.TensorVariable'> <class 'theano.tensor.var.TensorVariable'>


In [87]:
hidden = dense_in(c_ins[0][1])

In [125]:
for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [89]:
c_out = dense_out(hidden)

In [90]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [91]:
print(len(xs))

8


In [98]:
model.fit(xs, y, batch_size=64, nb_epoch=12)

Epoch 1/12
 1408/75110 [..............................] - ETA: 9s - loss: 1.6778

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


75110/75110 [==============================] - 9s 124us/step - loss: 1.7829
Epoch 2/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.7615
Epoch 3/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.7434
Epoch 4/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.7257
Epoch 5/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.7094
Epoch 6/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.6941
Epoch 7/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.6786
Epoch 8/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.6661
Epoch 9/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.6501
Epoch 10/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.6392
Epoch 11/12
75110/75110 [==============================] - 9s 124us/step - loss: 1.6301
Epoch 12/12
75110/75110 [===========================

In [99]:
get_next('for thos')

'e'

In [100]:
get_next('last che')

's'

In [101]:
get_next('queens a')

'n'

# First RNN with Keras

In [102]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, activation="relu", recurrent_initializer="identity")`
  app.launch_new_instance()


In [103]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_17 (Dense)             (None, 86)                22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [104]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


In [105]:
model.fit(np.concatenate(xs,axis=1), y, batch_size=64, nb_epoch=8)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/8
75110/75110 [==============================] - 11s 146us/step - loss: 2.7727
Epoch 2/8
75110/75110 [==============================] - 11s 146us/step - loss: 2.2619
Epoch 3/8
75110/75110 [==============================] - 11s 146us/step - loss: 2.0557
Epoch 4/8
75110/75110 [==============================] - 11s 146us/step - loss: 1.9177
Epoch 5/8
75110/75110 [==============================] - 11s 145us/step - loss: 1.8160
Epoch 6/8
75110/75110 [==============================] - 11s 145us/step - loss: 1.7343
Epoch 7/8
75110/75110 [==============================] - 11s 145us/step - loss: 1.6702
Epoch 8/8
75110/75110 [==============================] - 11s 145us/step - loss: 1.6184


In [107]:
def get_next_keras(inp):
    idxs = [char_to_index[c] for c in inp]
    arrs = np.array(idxs)[np.newaxis,:]
    p = model.predict(arrs)[0]
    return chars[np.argmax(p)]

In [108]:
get_next_keras('this is ')

't'

In [109]:
get_next_keras('part of ')

't'

In [110]:
get_next_keras('queens a')

'n'

# Returning Sequences

In [111]:
c_out_dat = [[indexes[i+n] for i in range(1, len(indexes)-cs, cs)] for n in range(cs)]

In [112]:
ys = [np.array(c[:-2]) for c in c_out_dat]

In [116]:
[c_in_dat[n][:cs] for n in range(cs)]


[[40, 1, 33, 2, 72, 67, 73, 2],
 [42, 1, 38, 44, 2, 9, 61, 73],
 [29, 43, 31, 71, 54, 9, 58, 61],
 [30, 45, 2, 74, 2, 76, 67, 58],
 [25, 40, 73, 73, 76, 61, 24, 71],
 [27, 40, 61, 61, 68, 54, 2, 58],
 [29, 39, 54, 2, 66, 73, 33, 2],
 [1, 43, 73, 62, 54, 2, 72, 67]]

In [117]:
[c_out_dat[n][:cs] for n in range(cs)]

[[42, 1, 38, 44, 2, 9, 61, 73],
 [29, 43, 31, 71, 54, 9, 58, 61],
 [30, 45, 2, 74, 2, 76, 67, 58],
 [25, 40, 73, 73, 76, 61, 24, 71],
 [27, 40, 61, 61, 68, 54, 2, 58],
 [29, 39, 54, 2, 66, 73, 33, 2],
 [1, 43, 73, 62, 54, 2, 72, 67],
 [1, 33, 2, 72, 67, 73, 2, 68]]

In [122]:
inp1 = Input(shape=(n_fac,), name = 'zeros')
hidden = dense_in(inp1)

In [126]:
outs = []
for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    outs.append(dense_out(hidden))

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [127]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [128]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(75110, 42)

In [129]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/12
75110/75110 [==============================] - 25s 328us/step - loss: 20.3479 - dense_16_loss_1: 2.6217 - dense_16_loss_2: 2.5420 - dense_16_loss_3: 2.4985 - dense_16_loss_4: 2.5365 - dense_16_loss_5: 2.5645 - dense_16_loss_6: 2.6120 - dense_16_loss_7: 2.6578 - dense_16_loss_8: 2.3148
Epoch 2/12
75110/75110 [==============================] - 25s 329us/step - loss: 13.6544 - dense_16_loss_1: 1.6049 - dense_16_loss_2: 1.5267 - dense_16_loss_3: 1.5098 - dense_16_loss_4: 1.5795 - dense_16_loss_5: 1.6158 - dense_16_loss_6: 1.6508 - dense_16_loss_7: 1.7438 - dense_16_loss_8: 2.4230
Epoch 3/12
75110/75110 [==============================] - 25s 328us/step - loss: 8.9874 - dense_16_loss_1: 0.8449 - dense_16_loss_2: 0.8359 - dense_16_loss_3: 0.8526 - dense_16_loss_4: 0.8845 - dense_16_loss_5: 0.9495 - dense_16_loss_6: 1.0340 - dense_16_loss_7: 1.1242 - dense_16_loss_8: 2.4619
Epoch 4/12
75110/75110 [==============================] - 25s 329us/step - loss: 6.4848 - dense_16_loss_1: 0.4

In [132]:
def get_nexts(inp):
    idxs = [char_to_index[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]


In [133]:
get_nexts(' this is')


[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'i', 's', ' ', 'i', 's', ' ']

In [134]:
get_nexts(' part of')


[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['p', 'a', 'r', 't', ' ', 'o', 'f', ' ']

# Sequence Model with Keras

In [135]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.summary()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, return_sequences=True, activation="relu", recurrent_initializer="identity")`
  app.launch_new_instance()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 8, 42)             3612      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 8, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 8, 86)             22102     
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
_________________________________________________________________


In [136]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [139]:
x_rnn=np.stack(np.squeeze(xs), axis=1)
y_rnn=np.atleast_3d(np.stack(ys, axis=1))

In [140]:
x_rnn.shape, y_rnn.shape

((75110, 8), (75110, 8, 1))

In [138]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 12s 164us/step - loss: 2.4288
Epoch 2/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.9992
Epoch 3/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.8845
Epoch 4/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.8246
Epoch 5/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.7861
Epoch 6/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.7586
Epoch 7/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.7384
Epoch 8/8
75110/75110 [==============================] - 12s 163us/step - loss: 1.7229


In [142]:
def get_nexts_keras(inp):
    idxs = [char_to_index[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]


In [143]:
get_nexts_keras(' this is')


[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 's', 's', ' ']

# One-Hot Sequence Keras Model

In [144]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())


/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(256, return_sequences=True, activation="relu", input_shape=(8, 86), recurrent_initializer="identity")`
  app.launch_new_instance()


In [156]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((75110, 8, 86), (75110, 8, 86))

In [157]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 10s 139us/step - loss: 2.4482
Epoch 2/8
75110/75110 [==============================] - 10s 138us/step - loss: 2.0428
Epoch 3/8
75110/75110 [==============================] - 10s 138us/step - loss: 1.9266
Epoch 4/8
75110/75110 [==============================] - 10s 138us/step - loss: 1.8603
Epoch 5/8
75110/75110 [==============================] - 10s 137us/step - loss: 1.8171
Epoch 6/8
75110/75110 [==============================] - 10s 137us/step - loss: 1.7862
Epoch 7/8
75110/75110 [==============================] - 10s 137us/step - loss: 1.7623
Epoch 8/8
75110/75110 [==============================] - 10s 137us/step - loss: 1.7435


In [158]:
def get_nexts_oh(inp):
    idxs = np.array([char_to_index[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [159]:
get_nexts_oh(' this is')

[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 'a', 's', ' ']

# Stateful Model with Keras

In [160]:
bs = 64

In [161]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])


In [162]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [163]:
mx = len(x_rnn)//bs*bs

In [174]:
len(x_rnn), mx

(75110, 75072)

In [164]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 35s 469us/step - loss: 2.2515
Epoch 2/4
75072/75072 [==============================] - 35s 465us/step - loss: 1.9768
Epoch 3/4
75072/75072 [==============================] - 35s 465us/step - loss: 1.8961
Epoch 4/4
75072/75072 [==============================] - 35s 465us/step - loss: 1.8483


In [175]:
get_nexts_keras(' this is')

ValueError: dimension mismatch in args to gemm (64,256)x(256,256)->(1,256)
Apply node that caused the error: GpuGemm{inplace}(GpuDot22.0, TensorConstant{0.20000000298}, lstm_1/variable[t-1][cuda], <CudaNdarrayType(float32, matrix)>, TensorConstant{0.20000000298})
Toposort index: 10
Inputs types: [CudaNdarrayType(float32, matrix), TensorType(float32, scalar), CudaNdarrayType(float32, matrix), CudaNdarrayType(float32, matrix), TensorType(float32, scalar)]
Inputs shapes: [(1, 256), (), (64, 256), (256, 256), ()]
Inputs strides: [(0, 1), (), (256, 1), (1024, 1), ()]
Inputs values: ['not shown', array(0.20000000298023224, dtype=float32), 'not shown', 'not shown', array(0.20000000298023224, dtype=float32)]
Outputs clients: [[GpuElemwise{Composite{(clip((i0 + i1 + i2), i3, i4) * tanh(i5))},no_inplace}(CudaNdarrayConstant{[[ 0.5]]}, GpuElemwise{mul,no_inplace}.0, GpuGemm{inplace}.0, CudaNdarrayConstant{[[ 0.]]}, CudaNdarrayConstant{[[ 1.]]}, GpuElemwise{Composite{((clip((i0 + i1 + i2), i3, i4)